In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import numpy as np
import jax
import jax.numpy as jnp
import flax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import timecast as tc

from mpl_toolkits import mplot3d

plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

/home/dsuo/miniconda3/envs/skgaip/lib/python3.7/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
import pickle

import timecast as tc
import pandas as pd
import matplotlib.pyplot as plt

import flax
import jax.numpy as jnp
import numpy as np

from timecast.learners import AR
from timecast.utils.ar import historify, compute_gram

In [3]:
data = jnp.array(pd.read_csv("../data/wind/original/MS_winds.dat", names=list(range(57))))
pickle.dump(data, open("../data/wind/original/MS_winds.pkl", "wb"))

In [4]:
# Columns = 57 stations
# Rows = wind speed readings (m/s)
data

DeviceArray([[5.0963, 2.0564, 3.0399, ..., 3.0399, 3.5763, 2.5481],
             [5.0963, 1.5199, 2.5481, ..., 2.5481, 3.5763, 2.5481],
             [5.588 , 1.5199, 2.0564, ..., 2.5481, 3.5763, 1.5199],
             ...,
             [4.6045, 4.0681, 5.0963, ..., 4.6045, 0.    , 3.0399],
             [7.1526, 6.1244, 4.6045, ..., 4.0681, 0.    , 4.0681],
             [7.1526, 3.5763, 3.0399, ..., 4.0681, 0.    , 4.6045]],            dtype=float32)

In [5]:
# Normalization
# NOTE: This is a bug; they claim this normalizes from 0 to 1, but it doesn't
# NOTE: Their variable also refer to min and max as mean and std, respectively so...
data_min = data.min()
data_max = data.max()
data = (data - data_min) / data_max

In [6]:
models = pickle.load(open("../data/wind/original/models.pkl", "rb"))["models"]

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [7]:
# Mostly from https://github.com/amirstar/Deep-Forecast/blob/4dcdf66f8ae3070ab706b30a6e3cf888f36e0536/multiLSTM.py#L210
def predict(X, models):
    X = X.reshape(X.shape[0], history_len, num_stations)
    results = np.zeros_like(np.zeros((X.shape[0], num_stations)))
    
    for ind in range(len(X)):
        modelInd = ind % 6
        if modelInd == 0:
            testInputRaw = X[ind]
            testInputShape = testInputRaw.shape
            testInput = np.reshape(testInputRaw, [1, testInputShape[0], testInputShape[1]])
        else:
            testInputRaw = np.vstack((testInputRaw, results[ind-1]))
            testInput = np.delete(testInputRaw, 0, axis=0)
            testInputShape = testInput.shape
            testInput = np.reshape(testInput, [1, testInputShape[0], testInputShape[1]])
    
        pred = models[modelInd].predict(testInput)
        results[ind] = pred

    return jnp.array(results)

In [8]:
num_train = 6000
num_test = 361

history_len = 12
num_stations = 57

In [9]:
# 12..5999
train_true = data[history_len:num_train]
# 0..11, 1..12, ..., 5987..5998
train_data = historify(data, history_len=history_len, num_histories=train_true.shape[0])

# 6012..8386
test_true = data[num_train + history_len:]
# 6000..6011, ..., 8374..8385
test_data = historify(data, history_len=history_len, num_histories=test_true.shape[0], offset=num_train)

In [10]:
# 6012..8386
test_pred = predict(test_data, models)

In [11]:
data.shape

(8387, 57)

In [12]:
# Metric: mean absolute error
jnp.absolute((test_true - test_pred) * data_max + data_min).mean(axis=0).mean()

DeviceArray(1.3113904, dtype=float32)

In [13]:
# 12..5999
train_pred = predict(train_data, models)

In [14]:
train_pred.shape

(5988, 57)

In [15]:
# 1..5998
train_pred = jnp.vstack((jnp.zeros((history_len - 1, num_stations)), train_pred))

In [16]:
print(test_pred.shape, test_data.shape)

(2375, 57) (2375, 684)


In [17]:
print(train_pred.shape, train_data.shape)

(5999, 57) (5988, 684)


In [18]:
ars, states = [None] * num_stations, [None] * num_stations
for station in tqdm.tqdm(range(num_stations)):
    ars[station], states[station] = AR.fit(
        data=[(data[:num_train - 1], train_pred[:, station], None)],
        input_dim=num_stations,
        output_dim=1,
        history=data[num_train : num_train + history_len],
        history_len=history_len
    )

In [19]:
for station in tqdm.tqdm(range(num_stations)):
    pickle.dump(ars[station].params, open("../data/wind/ar/{}.pkl".format(station), "wb"))

In [20]:
for station in tqdm.tqdm(range(num_stations)):
    pickle.dump(test_pred[:, station], open("../data/wind/base/{}.pkl".format(station), "wb"))

In [21]:
@tc.experiment("station", range(num_stations))
@tc.experiment("lr", jnp.linspace(-7, -4, 13))
@tc.experiment("history_len", [4, 8, 12, 16])
def runner(station, history_len, lr=-5):
    import jax.numpy as jnp
    import pickle
    
    from timecast.learners import Sequential, Parallel, BlackBox, AR
    from timecast import tmap
    from timecast.objectives import residual
    from timecast.optim import GradientDescent
    
    num_train = 6000

#     history_len = 4
    num_stations = 57

    data = jnp.asarray(pickle.load(open("../data/wind/original/MS_winds.pkl", "rb")))
    data_min = data.min()
    data_max = data.max()
    data = (data - data_min) / data_max
    
    Y_lstm = jnp.asarray(pickle.load(open("../data/wind/base/{}.pkl".format(station), "rb")))
#     params = pickle.load(open("../data/wind/ar/{}.pkl".format(station), "rb"))
    
    lstm = BlackBox.partial(arr=Y_lstm)
    ar = AR.partial(
        output_dim=1,
        history=data[num_train : num_train + history_len - 1],
        history_len=history_len
    )
    model, state = Parallel.new(shape=(1, num_stations), learners=[lstm, ar])
#     model.params["AR"] = params

    optim_def = GradientDescent(learning_rate=(10 ** lr))
    optimizer = optim_def.create(model)

    X = data[num_train + history_len - 1:-1]
    Y = data[num_train + history_len:, station]

    Y_hat, optimizer, state = tmap(X, Y, optimizer, state=state, objective=residual)
    
    return {
        "station": station,
        "lr": lr,
        "history_len": history_len,
        "mae": jnp.square((Y - Y_hat) * data_max + data_min).mean()
    }

In [22]:
results = runner.run(processes=15, tqdm=tqdm)

In [23]:
df = {}
for lr in jnp.linspace(-7, -4, 13):
    df[lr] = {}
    for history_len in [4, 8, 12, 16]:
        df[lr][history_len] = np.mean([result["mae"] for result in results if (result["lr"] == lr and result["history_len"] == history_len)])

In [24]:
df = pd.DataFrame.from_records(df)

In [25]:
df

,-7.00,-6.75,-6.50,-6.25,-6.00,-5.75,-5.50,-5.25,-5.00,-4.75,-4.50,-4.25,-4.00
4,6.803426,6.802893,6.801956,6.800316,6.797490,6.792729,6.785058,6.773670,6.759305,6.747010,6.747928,6.772299,6.814532
8,6.808669,6.807908,6.806577,6.804286,6.800442,6.794290,6.785281,6.774267,6.765826,6.769637,6.794618,6.837570,6.890489
12,6.814820,6.813985,6.812538,6.810084,6.806076,6.799983,6.791938,6.784324,6.783747,6.799695,6.836451,6.890458,6.966245
16,6.821484,6.820539,6.818916,6.816198,6.811869,6.805592,6.798119,6.793053,6.798106,6.821633,6.865398,6.928493,7.021152
